<a href="https://qworld.net" target="_blank" align="left"><img src="../qworld/images/header.jpg"  align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b>Quantum Tomography  </b></font>
<br>
_prepared by Abuzer Yakaryilmaz_
<br><br>
[<img src="../qworld/images/watch_lecture.jpg" align="left">](https://youtu.be/mIEiWCJ6R58)
<br><br><br>

We study a simplified version of quantum tomography here. 

It is similar to learn the bias of a coin by collecting statistics from tossing this coin many times. But, only making measurement may not be enough to make a good guess.

Suppose that you are given 1000 copies of a qubit and your task is to learn the state of this qubit. We use a python class called "unknown_qubit" for doing our quantum experiments. 

Please run the following cell before continuing.

In [1]:
# class unknown_qubit
#   available_qubit = 1000 -> you get at most 1000 qubit copies
#   get_qubits(number_of_qubits) -> you get the specified number of qubits for your experiment
#   measure_qubits() -> your qubits are measured and the result is returned as a dictionary variable
#                    -> after measurement, these qubits are destroyed
#   rotate_qubits(angle) -> your qubits are rotated with the specified angle in radian
#   compare_my_guess(my_angle) -> your guess in radian is compared with the real angle

from random import randrange
from math import pi, acos
import pennylane as qml

class unknown_qubit:
    def __init__(self):
        self.__theta = randrange(18000)/18000*pi        
        self.__available_qubits = 1000
        self.__active_qubits = 0
        self.__rotations = 0  # cumulative rotation angle
        self.__state_prepared = False
        print(self.__available_qubits, "qubits are created")

        # Use default qubit device, single qubit, shots will be specified later
        self.dev = qml.device("default.qubit", wires=1, shots=None)  # shots set dynamically
        
    def get_qubits(self, number_of_qubits=None):
        if number_of_qubits is None or not isinstance(number_of_qubits, int) or number_of_qubits < 1:
            print("\nERROR: the method 'get_qubits' takes the number of qubit(s) as a positive integer, i.e., get_qubits(100)")
            return
        if number_of_qubits <= self.__available_qubits:
            self.__active_qubits = number_of_qubits
            self.__available_qubits -= self.__active_qubits
            self.__rotations = 0
            self.__state_prepared = True
            print(f"\nYou have {number_of_qubits} active qubits that are set to (cos(theta), sin(theta))")
            self.available_qubits()
        else:
            print(f"\nWARNING: you requested {number_of_qubits} qubits, but there is not enough available qubits!")
            self.available_qubits()

    def _circuit(self):
        # Circuit prepares the initial unknown qubit state and applies cumulative rotations
        qml.RY(2 * self.__theta, wires=0)
        if self.__rotations != 0:
            qml.RY(2 * self.__rotations, wires=0)

    def measure_qubits(self):
        if self.__active_qubits > 0 and self.__state_prepared:
            # Define a QNode for measurement with shots = active qubits
            @qml.qnode(self.dev)
            def circuit():
                self._circuit()
                return qml.sample(qml.PauliZ(0))  # sample measurement in Z basis
            
            # Run the circuit with shots equal to active qubits
            self.dev.shots = self.__active_qubits
            results = circuit()
            # results are +1 or -1; +1 corresponds to |0>, -1 corresponds to |1>
            counts = {'0': int((results == 1).sum()), '1': int((results == -1).sum())}
            
            print(f"\nYour {self.__active_qubits} qubits are measured")
            print("counts = ", counts)
            self.__active_qubits = 0
            self.__state_prepared = False
            return counts
        else:
            print("\nWARNING: there is no active qubits -- you might first execute 'get_qubits()' method")
            self.available_qubits()

    def rotate_qubits(self, angle=None):
        if angle is None or (not isinstance(angle, float) and not isinstance(angle, int)):
            print("\nERROR: the method 'rotate_qubits' takes a real-valued angle in radian as its parameter, i.e., rotate_qubits(1.2121)")
        elif self.__active_qubits > 0 and self.__state_prepared:
            # Update cumulative rotation angle
            self.__rotations += angle
            print(f"\nYour active qubits are rotated by angle {angle} in radian")
        else:
            print("\nWARNING: there is no active qubits -- you might first execute 'get_qubits()' method")
            self.available_qubits()

    def compare_my_guess(self, my_angle):
        if my_angle is None or (not isinstance(my_angle, float) and not isinstance(my_angle, int)):
            print("ERROR: the method 'compare_my_guess' takes a real-valued angle in radian as your guessed angle, i.e., compare_my_guess(1.2121)")
        else:
            self.__available_qubits = 0
            diff = abs(my_angle - self.__theta)
            print(f"\n{self.__theta} is the original")
            print(f"{my_angle} is your guess")
            print(f"the angle difference between the original theta and your guess is {diff/pi*180} degree")
            print("--> the number of available qubits is (set to) zero, and so you cannot make any further experiment")

    def available_qubits(self):
        print(f"--> the number of available unused qubit(s) is {self.__available_qubits}")
             


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\prami\OneDrive - University of Witwatersrand\External projects\stellies\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\prami\OneDrive - University of Witwatersrand\External projects\stellies\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\prami\OneDrive - Universi

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



class unknown_qubit:
    
    available_qubit = 1000 -> you get at most 1000 qubit copies
    get_qubits(number_of_qubits) -> you get the specified number of qubits for your experiment
    measure_qubits() -> your qubits are measured and the result is returned as a dictionary variable
                     -> after measurement, these qubits are destroyed
    rotate_qubits(angle) -> your qubits are rotated with the specified angle in radian
    compare_my_guess(my_angle) -> your guess in radian is compared with the real angle

<h3> Task 1 </h3>

You are given 1000 copies of the identical qubits which are in the same quantum state lying in the first or second quadrant of the unit circle. 

This quantum state is represented by an angle $ \theta \in [0,\pi) $, and your task is to guess this angle.

You use the class __unknown_qubit__ and its methods for your experiments. 

_Remark that the measurement outcomes of the quantum states with angles $ \pi \over 3 $ and $ 2 \pi \over 3 $ are identical even though they are different quantum states. Therefore, getting 1000 qubits and then measuring them does not guarantee the correct answer._

Test your solution at least ten times.

In [2]:
from math import pi, cos, sin, acos, asin

# an angle theta is randomly picked and it is fixed througout the experiment
# my_experiment = unknown_qubit() 
#
# my_experiment.get_qubits(number_of_qubits)
# my_experiment.rotate_qubits(angle)
# my_experiment.measure_qubits()
# my_experiment.compare_my_guess(my_angle)
#

from math import pi, acos

my_experiment = unknown_qubit()

# Use 900 copies to determine candidates
my_experiment.get_qubits(900)
counts = my_experiment.measure_qubits()

number_of_observed_zeros = counts.get('0', 0)
probability_of_zeros = number_of_observed_zeros / 900
cos_theta = probability_of_zeros ** 0.5
theta = acos(cos_theta)

theta_first_candidate = theta
theta_second_candidate = pi - theta

print("The first candidate is", theta_first_candidate, "radians,", theta_first_candidate * 180/pi, "degrees")
print("The second candidate is", theta_second_candidate, "radians,", theta_second_candidate * 180/pi, "degrees")

# Use remaining 100 copies to test which candidate is better
my_experiment.get_qubits(100)
my_experiment.rotate_qubits(-theta_first_candidate)

counts = my_experiment.measure_qubits()
number_of_observed_zeros = counts.get('0', 0)

if number_of_observed_zeros == 100:
    my_guess = theta_first_candidate
else:
    my_guess = theta_second_candidate

my_experiment.compare_my_guess(my_guess)



1000 qubits are created

You have 900 active qubits that are set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 100


AttributeError: Shots can no longer be set on a device instance. You can set shots on a call to a QNode, on individual tapes, or create a new device instance instead.

In [3]:
for i in range(10):
    print(f"Experiment {i+1}")
    print("___________\n")
    my_experiment = unknown_qubit()
    my_experiment.get_qubits(900)
    counts = my_experiment.measure_qubits()

    number_of_observed_zeros = counts.get('0', 0)
    probability_of_zeros = number_of_observed_zeros / 900
    cos_theta = probability_of_zeros ** 0.5
    theta = acos(cos_theta)

    theta_first_candidate = theta
    theta_second_candidate = pi - theta
    
    my_experiment.get_qubits(100)
    my_experiment.rotate_qubits(-theta_first_candidate)

    counts = my_experiment.measure_qubits()
    number_of_observed_zeros = counts.get('0', 0)

    if number_of_observed_zeros == 100:
        my_guess = theta_first_candidate
    else:
        my_guess = theta_second_candidate

    my_experiment.compare_my_guess(my_guess)
    print("\n\n")


Experiment 1
___________

1000 qubits are created

You have 900 active qubits that are set to (cos(theta), sin(theta))
--> the number of available unused qubit(s) is 100


AttributeError: Shots can no longer be set on a device instance. You can set shots on a call to a QNode, on individual tapes, or create a new device instance instead.

<h3> Task 3 (Discussion) </h3>

If the angle in Task 1 is picked in range $ [0,2\pi) $, then can we determine its quadrant correctly?

<h3> Global phase </h3>

Suppose that we have a qubit and its state is either $ \ket{0} $ or $ -\ket{0} $.

Is there any sequence of one-qubit gates such that we can measure different results after applying them?

All one-qubit gates are $ 2 \times 2 $ matrices, and their application is represented by a single matrix: $ A_n \cdot \cdots \cdot A_2 \cdot A_1 = A $.

By linearity, if $ A \ket{0} = \ket{u} $, then $ A (- \ket{0}) = -\ket{u} $. Thus, after measurement, the probabilities of observing state $ \ket{0} $ and state $ \ket{1} $ are the same for $ \ket{u} $ and $ -\ket{u} $. Therefore, we cannot distinguish them.

Even though the states $ \ket{0} $ and $ -\ket{0} $ are different mathematically, they are assumed as identical from the physical point of view. 

The minus sign in front of $ -\ket{0} $ is called as a global phase.

In general, a global phase can be a complex number with magnitude 1.